In [4]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model='gpt-4o')
result = model.invoke("Tell me something about Evaluator Optimizer with Langgraph with syntax")
result

AIMessage(content='"Evaluator Optimizer" and "Langgraph" do not appear to be widely recognized or established concepts, products, or frameworks in the public domain as of my last update in October 2023. It\'s possible that they are newly introduced tools, proprietary technologies, or hypothetical terms in development or niche use.\n\nIf you\'re referring to a specific technology stack or a new feature in a development tool, I recommend checking the official documentation or resources associated with the respective technology or framework. Typically, these resources provide the most accurate and up-to-date syntax examples and use cases.\n\nIf you can provide more context or specify whether these terms are related to a particular programming language, software tool, or domain (like machine learning, data processing, or natural language processing), I might be able to offer more targeted assistance or suggestions.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'

In [5]:
from typing import Annotated, List
import operator
from typing_extensions import Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from typing_extensions import TypedDict

class State(TypedDict):
    joke:str
    topic:str
    feedback:str
    funny_or_not: str


In [6]:
class Feedback(BaseModel):
    grade: Literal["funny", "not funny"] = Field(
        description="Decide if the joke is funny or not",
    )
    feedback: str = Field(
        description="If the joke is not funny, provide the feedback on how to improve it."
    )

evaluator = model.with_structured_output(Feedback)

In [8]:
def llm_call_generator(state:State):
    """LLM generates a joke"""
    if state.get("feedback"):
        msg = model.invoke(
            f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}"
        )
    else:
        msg = model.invoke(
            f"Write a joke about {state['topic']}"
        )
    return {"joke": msg.content}

def llm_call_evaluator(state:State):
    """LLM evaluates the joke"""

    grade = evaluator.invoke(f"Grade teh joke {state['joke']}")
    return {"funny_or_not": grade.grade, "feedback":grade.feedback}

def route_joke(state:State):
    """Route back to joke generator or end based on feedback from the evaluator"""

    if state["funny_or_not"] == "funny":
        return "Accepted"
    elif state["funny_or_not"] == "not_funny":
        return "Rejected + Feedback"


In [10]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(State)

#Add nodes
workflow.add_node("llm_call_generator", llm_call_generator)
workflow.add_node("llm_call_evaluator", llm_call_evaluator)

#Add edges
workflow.add_edge(START, "llm_call_generator")
workflow.add_edge("llm_call_generator", "llm_call_evaluator")
workflow.add_conditional_edges(
    "llm_call_evaluator",
    route_joke,
    { #Name returned by route_joke: Namme of next node to visit
        "Accepted":END,
        "Rejected + Feedback":"llm_call_generator",

    }
)

optimized_workflow = workflow.compile()

In [11]:
state = optimized_workflow.invoke({"topic":"Agentic AI SYstems"})
print(state["joke"])

Why did the agentic AI system apply for a job at the bakery?

Because it heard they kneaded a rise in self-awareness!
